<a href="https://colab.research.google.com/github/meganflinders/deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization2_h5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [22]:
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [23]:
application_df = application_df.drop(columns=['EIN', 'NAME', 'USE_CASE', 'STATUS', 'SPECIAL_CONSIDERATIONS'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,Association,0,5000,1
1,T3,Independent,C2000,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,Association,0,5000,0
3,T3,CompanySponsored,C2000,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Trust,100000-499999,142590,1


In [24]:
application_type_values = application_df['APPLICATION_TYPE'].value_counts()
application_type_values

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [25]:
application_types_to_replace = application_type_values[application_type_values < 1000].index

for app in application_types_to_replace:
  application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, 'Other')

application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [26]:
classification_values = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = classification_values[classification_values < 1000].index

for cls in classifications_to_replace:
  application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,'Other')

application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [27]:
preprocessed_df = pd.get_dummies(application_df, columns=['APPLICATION_TYPE', 'AFFILIATION','CLASSIFICATION', 'ORGANIZATION', 'INCOME_AMT'])
preprocessed_df.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,108590,1,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,5000,0,0,0,0,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,0
3,6692,1,0,0,1,0,0,0,1,0,...,1,0,0,1,0,0,0,0,0,0
4,142590,1,0,0,1,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [28]:
y = preprocessed_df['IS_SUCCESSFUL']
X = preprocessed_df.drop(columns='IS_SUCCESSFUL')

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [30]:
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)

In [31]:
X_train_scaled =X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [44]:
nn = tf.keras.models.Sequential()
nn.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim = 32))
nn.add(tf.keras.layers.Dropout(0.2))
#nn.add(tf.keras.layers.BatchNormalization())
nn.add(tf.keras.layers.Dense(units=64, activation='relu'))
nn.add(tf.keras.layers.Dense(units=128, activation='relu'))
#nn.add(tf.keras.layers.Dense(units=128, activation='relu'))
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 32)                1056      
                                                                 
 dropout_5 (Dropout)         (None, 32)                0         
                                                                 
 dense_20 (Dense)            (None, 64)                2112      
                                                                 
 dense_21 (Dense)            (None, 128)               8320      
                                                                 
 dense_22 (Dense)            (None, 1)                 129       
                                                                 
Total params: 11,617
Trainable params: 11,617
Non-trainable params: 0
_________________________________________________________________


In [45]:
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [46]:
trained_model = nn.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5877 - accuracy: 0.7075
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5698 - accuracy: 0.7199
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5665 - accuracy: 0.7221
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5646 - accuracy: 0.7226
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5630 - accuracy: 0.7224
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5616 - accuracy: 0.7243
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5615 - accuracy: 0.7255
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5612 - accuracy: 0.7236
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5611 - accuracy: 0.7239
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5595 - accura

In [47]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5598 - accuracy: 0.7256 - 463ms/epoch - 2ms/step
Loss: 0.5597580671310425, Accuracy: 0.7255976796150208


In [48]:
nn.save('AlphabetSoupCharity_Optimization2.h5')